In [1]:
import pickle
import numpy as np
import os


## Function Definitions

In [2]:
def createColorSequence(df):
    colors = []
    for i in df['backend']:
        if i == 'tflite_edgetpu':
            colors.append('orange')
        elif i == 'tf_cpu':
            colors.append('red')
        else: # GPU case
            colors.append('blue')
    return colors

In [5]:
def createColorSequence_advanced(df):
    colors = []
    for index, row in df.iterrows():
        if row['architecture'] == 'Core i7-5650U' & row['backend'] == 'tflite_edgetpu':
            colors.append('orange')
        elif row['architecture'] == 'Core i7-5650U' & row['backend'] == 'tf_cpu':
            colors.append('red')
        if row['architecture'] == 'Core i7-6700' & row['backend'] == 'tflite_edgetpu':
            colors.append('dark orange')
        elif row['architecture'] == 'Core i7-6700' and row['backend'] == 'tf_cpu':
            colors.append('dark red')
        else:
            print(row['architecture'])
            print(row['backend'])
            colors.append('grey')

    return colors

In [4]:
createColorSequence_advanced(energy_df)

NameError: name 'energy_df' is not defined

In [7]:
def getFamilyName(model_name):
    if 'densenet' in model_name.lower():
        model_family='DenseNet'
    elif 'inceptionresnet' in model_name.lower():
        model_family='InceptionResNet'
    elif 'efficientnet' in model_name.lower():
        model_family='EfficientNet'
    elif 'inception' in model_name.lower():
        model_family='Inception'
    elif 'mobilenet' in model_name.lower():
        model_family =  'MobileNet'
    elif 'nasnet' in model_name.lower():
        model_family =  'NASNet'
    elif 'resnet' in model_name.lower():
        model_family =  'ResNet'
    elif 'vgg' in model_name.lower():
        model_family = 'VGG'
    elif 'xception' in  model_name.lower():
        model_family = 'Xception'
    elif 'yolo8' in model_name.lower():
        model_family = 'YOLO8'
    else :
        model_family = ''
    return model_family

In [6]:
def approx_USB_power_draw(row):
    if row['backend'] == 'tflite_edgetpu':
        return row['power_draw'] + 25 * (row['running_time']/3600) # assume constant 25 watt
    else:
        return row['power_draw'] 

## Load Data 

In [71]:
with open(os.getcwd()+'/result_databases/eval6400_23_08_2023_mac.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data.shape)

with open(os.getcwd()+'/result_databases/eval6400_24_08_2023_mac.pkl', 'rb') as f:
    data = data.append( pickle.load(f) )
    print(data.shape)
with open(os.getcwd()+'/result_databases/eval3200_23_08_2023_mac.pkl', 'rb') as f:
   data = data.append( pickle.load(f) )
   print(data.shape)

with open(os.getcwd()+'/result_databases/eval3200_23_08_2023_mac.pkl', 'rb') as f:
   data = data.append( pickle.load(f) )
   print(data.shape)

with open(os.getcwd()+'/result_databases/eval12000_16_08_2023_pyrapl_1.pkl', 'rb') as f:
   data = data.append( pickle.load(f) )
   print(data.shape)

with open(os.getcwd()+'/result_databases/eval12000_16_08_2023_pyrapl_2.pkl', 'rb') as f:
   data = data.append( pickle.load(f) )

#data = data.groupby(['model','backend']).mean()#.reset_index()

#Add Family of 
data['family']= data['model'].apply(lambda a : getFamilyName(a))
data['approx_USB_power_draw'] = data.apply(lambda row: approx_USB_power_draw(row), axis = 1)
filtered_data = data[['dataset', 'model', 'backend', 'software',
       'architecture', 'number_of_operations', 'number_of_unmapped_operations',
       'input_shape', 'total_parameters', 'trainable_parameters',
       'non_trainable_parameters', 'running_time', 'power_draw']]

data.columns

(30, 21)
(60, 21)
(90, 21)
(120, 21)
(150, 21)


/var/folders/0x/0mxpptjd20x3bk4b9bhllszr0000gn/T/ipykernel_10650/1513159506.py:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/0x/0mxpptjd20x3bk4b9bhllszr0000gn/T/ipykernel_10650/1513159506.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/0x/0mxpptjd20x3bk4b9bhllszr0000gn/T/ipykernel_10650/1513159506.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/0x/0mxpptjd20x3bk4b9bhllszr0000gn/T/ipykernel_10650/1513159506.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/0x/0mxpptjd20x3bk4b9bhllszr0000gn/T/ipykernel_10650/1513159506.py:21: FutureWarning:

The frame.append method is depr

Index(['log_name', 'task', 'dataset', 'model', 'backend', 'software',
       'architecture', 'number_of_operations', 'number_of_unmapped_operations',
       'input_shape', 'total_parameters', 'trainable_parameters',
       'non_trainable_parameters', 'running_time', 'power_draw',
       'validation_size', 'batch_size', 'accuracy_k1', 'accuracy_k3',
       'accuracy_k5', 'accuracy_k10', 'family', 'approx_USB_power_draw'],
      dtype='object')

## Average Runtimes per Environment

In [91]:
analyze_running_time= data[['model','backend','running_time','architecture']].dropna()#data[['model','backend','total_parameters', 'running_time','power_draw','accuracy','number_of_unmapped_operations']]

#analyze_power_draw = analyze_power_draw.pivot_table(index='model',columns = ['backend','architecture'], values='approx_USB_power_draw', aggfunc = 'mean')#set_index('model').wide_to_long()
analyze_running_time = analyze_running_time.pivot_table(index='backend', columns = ['architecture'], values='running_time', aggfunc = 'mean')#set_index('model').wide_to_long()

print(analyze_running_time)
#print('EDGETPU: ' )
#print(np.mean(analyze_power_draw['tflite_edgetpu', 'Core i7-5650U']))
#print('CPU: ')
#print(np.mean(analyze_power_draw['tf_cpu']))
#print('GPU')
#print(np.mean(analyze_power_draw['tf_gpu']))


architecture    Core i7-5650U  Core i7-6700
backend                                    
tf_cpu               0.126582      0.056629
tflite_edgetpu       0.049287      0.038730


## Average Power Draw

In [13]:
analyze_power_draw = data[['model','backend','approx_USB_power_draw','architecture']].dropna()#data[['model','backend','total_parameters', 'running_time','power_draw','accuracy','number_of_unmapped_operations']]

#analyze_power_draw = analyze_power_draw.pivot_table(index='model',columns = ['backend','architecture'], values='approx_USB_power_draw', aggfunc = 'mean')#set_index('model').wide_to_long()
analyze_power_draw = analyze_power_draw.pivot_table(index='backend',columns = ['architecture'], values='approx_USB_power_draw', aggfunc = 'mean')#set_index('model').wide_to_long()

print(analyze_power_draw)


architecture    Core i7-5650U  Core i7-6700
backend                                    
tf_cpu               1.888913      3.193612
tflite_edgetpu       0.260701      0.599567
EDGETPU: 
CPU: 


In [56]:
#data[['model','backend','accuracy_k1','family']].groupby(['family','backend']).mean()
data[['model','backend','accuracy_k1','family','architecture']].groupby(['backend']).mean()

/var/folders/0x/0mxpptjd20x3bk4b9bhllszr0000gn/T/ipykernel_4477/267296143.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data[['model','backend','accuracy_k1','family','architecture']].groupby(['backend']).mean()


,accuracy_k1
backend,
tf_cpu,0.693567
tflite_edgetpu,0.656326


## Plotly-Plots

In [94]:
import pandas as pd 
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

orange_colour = '#FEAD00'
light_orange_colour = '#FADFA4'

red_colour = '#C40000'
light_red_colour = '#FE6464'


### Laufzeit und Energieverbrauch Mac, Workstation für CPU und EdgeTPU 

In [96]:
rt_filtered= data[['model','backend','running_time','architecture']].dropna()#data[['model','backend','total_parameters', 'running_time','power_draw','accuracy','number_of_unmapped_operations']]
analyze_running_time = rt_filtered.pivot_table(index='backend',columns = ['architecture'], values='running_time', aggfunc = 'mean')#set_index('model').wide_to_long()
analyze_running_time_std = rt_filtered.pivot_table(index='backend',columns = ['architecture'], values='running_time', aggfunc = np.std)#set_index('model').wide_to_long()
#print(analyze_running_time_std)

ywerte = [analyze_running_time['Core i7-5650U']['tf_cpu'],analyze_running_time['Core i7-5650U']['tflite_edgetpu'],  analyze_running_time['Core i7-6700']['tf_cpu'],analyze_running_time['Core i7-6700']['tflite_edgetpu']]
errors = [analyze_running_time_std['Core i7-5650U']['tf_cpu'],analyze_running_time_std['Core i7-5650U']['tflite_edgetpu'],  analyze_running_time_std['Core i7-6700']['tf_cpu'],analyze_running_time_std['Core i7-6700']['tflite_edgetpu']]

xwerte = ['Mac: CPU','Mac: EdgeTPU', 'WS: CPU','WS: EdgeTPU']


fig = go.Figure(data=[go.Bar(x=xwerte, y=ywerte,
            hovertext=['CPU/EdgeTPU Laufzeitmessung'])])
# Customize aspect
fig.update_traces( marker_color=[light_red_colour,light_orange_colour,red_colour,orange_colour], marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6, error_y=dict(type='data', array = errors))

fig.update_layout(title_text='Laufzeit (ms)')
fig.show()

In [95]:
pd_filtered = data[['model','backend','approx_USB_power_draw','architecture']].dropna()#data[['model','backend','total_parameters', 'running_time','power_draw','accuracy','number_of_unmapped_operations']]

#analyze_power_draw = analyze_power_draw.pivot_table(index='model',columns = ['backend','architecture'], values='approx_USB_power_draw', aggfunc = 'mean')#set_index('model').wide_to_long()
analyze_power_draw = pd_filtered.pivot_table(index='backend',columns = ['architecture'], values='approx_USB_power_draw', aggfunc = 'mean')#set_index('model').wide_to_long()
analyze_power_draw_std = pd_filtered.pivot_table(index='backend',columns = ['architecture'], values='approx_USB_power_draw', aggfunc = np.std)#set_index('model').wide_to_long()




ywerte = [analyze_power_draw['Core i7-5650U']['tf_cpu'],analyze_power_draw['Core i7-5650U']['tflite_edgetpu'],  analyze_power_draw['Core i7-6700']['tf_cpu'],analyze_power_draw['Core i7-6700']['tflite_edgetpu']] #,np.mean(analyze_running_time['tf_gpu'])
errors = [analyze_power_draw_std['Core i7-5650U']['tf_cpu'],analyze_power_draw_std['Core i7-5650U']['tflite_edgetpu'],  analyze_power_draw_std['Core i7-6700']['tf_cpu'],analyze_power_draw_std['Core i7-6700']['tflite_edgetpu']]
xwerte = ['Mac: CPU','Mac: EdgeTPU', 'WS: CPU','WS: EdgeTPU']

fig = go.Figure(data=[go.Bar(x=xwerte, y=ywerte,
            hovertext=['CPU/EdgeTPU Energiemessung (Ws)'])])
# Customize aspect
fig.update_traces( marker_color=[light_red_colour,light_orange_colour,red_colour,orange_colour], marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6, error_y=dict(type='data', array = errors))

fig.update_layout(title_text='Energieverbrauch Ws')
fig.show()

# Only Model/Backend

### Energieverbrauch/Laufzeit pro Modell und Backend

In [93]:

energy_df = data[data['approx_USB_power_draw'].notna() ]
x_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['running_time'].aggregate('mean')

names_cpu =  list(energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu =  list(energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
y_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')

fig = go.Figure()
for i in range(x_cpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu[i]], y=[y_cpu[i]],
                        mode='markers',
                        name=str(names_cpu[i]),
                        marker = dict(color = 'red')))
for i in range(x_etpu.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu[i]], y=[y_etpu[i]],
                        mode='markers',
                        name=str(names_etpu[i]),
                        marker = dict(color = '#FEAD00')))
fig.update_layout(
    xaxis_title="Running Time",
    yaxis_title="Approx. Power Draw",
)

fig.show()


## k_1 Accuracy und Running Time

In [84]:
energy_df = data[data['running_time'].notna()]


x_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['running_time'].aggregate('mean')


y_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')
y_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')

names_cpu =  list(energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu = list(energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'], group_keys=True).groups.keys())
fig = go.Figure()
for i in range(x_cpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu[i]], y=[y_cpu[i]],
                        mode='markers',
                        name=str(names_cpu[i]),
                        marker = dict(color = 'red')))
for i in range(x_etpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_etpu[i]], y=[y_etpu[i]],
                        mode='markers',
                        name=str(names_etpu[i]),
                        marker = dict(color = 'orange')))
fig.update_layout(
    xaxis_title="Running Time",
    yaxis_title="Accuracy K=1",
)
fig.show()

## k_5 Accuracy und Running Time

In [85]:
energy_df = data[data['running_time'].notna()]


x_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['running_time'].aggregate('mean')


y_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['accuracy_k5'].aggregate('mean')
y_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['accuracy_k5'].aggregate('mean')

names_cpu =  list(energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu = list(energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'], group_keys=True).groups.keys())
fig = go.Figure()
for i in range(x_cpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu[i]], y=[y_cpu[i]],
                        mode='markers',
                        name=str(names_cpu[i]),
                        marker = dict(color = 'red')))
for i in range(x_etpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_etpu[i]], y=[y_etpu[i]],
                        mode='markers',
                        name=str(names_etpu[i]),
                        marker = dict(color = 'orange')))
fig.update_layout(
    xaxis_title="Running Time",
    yaxis_title="Accuracy K=5",
)
fig.show()

## K_1 Accuracy and Energy Consumption

In [88]:
energy_df = data[data['approx_USB_power_draw'].notna()]


x_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
x_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')


y_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')
y_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')

names_cpu =  list(energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu = list(energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'], group_keys=True).groups.keys())
fig = go.Figure()
for i in range(x_cpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu[i]], y=[y_cpu[i]],
                        mode='markers',
                        name=str(names_cpu[i]),
                        marker = dict(color = 'red')))
for i in range(x_etpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_etpu[i]], y=[y_etpu[i]],
                        mode='markers',
                        name=str(names_etpu[i]),
                        marker = dict(color = 'orange')))
fig.update_layout(
    xaxis_title="Approx Power Draw Ws",
    yaxis_title="Accuracy K=1",
)
fig.show()

## K_5 Accuracy und Energy Consumption

In [89]:
energy_df = data[data['approx_USB_power_draw'].notna()]


x_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
x_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')


y_cpu = energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'])['accuracy_k5'].aggregate('mean')
y_etpu = energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'])['accuracy_k5'].aggregate('mean')

names_cpu =  list(energy_df[energy_df['backend'] == 'tf_cpu'].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu = list(energy_df[energy_df['backend'] == 'tflite_edgetpu'].groupby(['model','backend'], group_keys=True).groups.keys())
fig = go.Figure()
for i in range(x_cpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu[i]], y=[y_cpu[i]],
                        mode='markers',
                        name=str(names_cpu[i]),
                        marker = dict(color = 'red')))
for i in range(x_etpu.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_etpu[i]], y=[y_etpu[i]],
                        mode='markers',
                        name=str(names_etpu[i]),
                        marker = dict(color = 'orange')))
fig.update_layout(
    xaxis_title="Approx Power Draw Ws",
    yaxis_title="Accuracy K=5",
)
fig.show()

In [127]:
import plotly.graph_objects as go

energy_df = data#[data['backend']=='tflite_edgetpu']

traces = []

for modelname in energy_df['model'].unique():
    colorlist = list(createColorSequence_advanced(energy_df[energy_df['model']==modelname]))
    x = energy_df[energy_df['model']==modelname]['running_time']
    y = energy_df[energy_df['model']==modelname]['accuracy_k1']
    traces.append(go.Scatter(x = x,
                             y = y , 
                             marker=dict(color=colorlist ), 
                             name = modelname, 
                             mode = 'markers',
                             visible='legendonly'
                             ))
    
   
layout = go.Layout(title=dict(text='Netz auswählen',x=0.5))
   
fig = go.Figure(data=traces, layout=layout)

max_rt = energy_df.loc[energy_df['running_time'].idxmax()]
min_rt = energy_df.loc[energy_df['running_time'].idxmin()]

fig.update_layout(width=800, yaxis=dict(title_text = 'Accuracy', range=[0, 1]), xaxis=dict(title_text = 'Running Time', range = [min_rt,max_rt])) 

fig.show()


Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
nan
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-5650U
tf_cpu
Core i7-5650U
tflite_edgetpu
Core i7-565

# Differentiate by Model/CPU or EDGTE TPU/Environment

## Accuracy K = 1 / Runtime

In [111]:

energy_df = data[data['running_time'].notna() ]
x_cpu_mac = energy_df[(energy_df['backend'] == 'tf_cpu' )& (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu_mac = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['running_time'].aggregate('mean')

names_cpu_mac =  list(energy_df[(energy_df['backend'] == 'tf_cpu' )& (energy_df['architecture'] =='Core i7-5650U' )].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu_mac =  list(energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu_mac = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')
y_etpu_mac = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')


x_cpu_ws = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu_ws = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['running_time'].aggregate('mean')

names_cpu_ws =  list(energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu_ws =  list(energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu_ws = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')
y_etpu_ws = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')


fig = go.Figure()
for i in range(x_cpu_mac.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu_mac[i]], y=[y_cpu_mac[i]],
                        mode='markers',
                        name=str(names_cpu_mac[i]),
                        marker = dict(color = light_red_colour)))
for i in range(x_etpu_mac.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu_mac[i]], y=[y_etpu_mac[i]],
                        mode='markers',
                        name=str(names_etpu_mac[i]),
                        marker = dict(color = light_orange_colour)))
    
for i in range(x_cpu_ws.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu_ws[i]], y=[y_cpu_ws[i]],
                        mode='markers',
                        name=str(names_cpu_ws[i]),
                        marker = dict(color = red_colour)))
for i in range(x_etpu_ws.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu_ws[i]], y=[y_etpu_ws[i]],
                        mode='markers',
                        name=str(names_etpu_ws[i]),
                        marker = dict(color = orange_colour)))
fig.update_layout(
    xaxis_title="Running Time",
    yaxis_title="Accuracy K=1",
)

fig.show()


## Accuracy K = 1 / Energy Consumption

In [112]:

energy_df = data[data['approx_USB_power_draw'].notna() ]
x_cpu_mac = energy_df[(energy_df['backend'] == 'tf_cpu' )& (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
x_etpu_mac = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')

names_cpu_mac =  list(energy_df[(energy_df['backend'] == 'tf_cpu' )& (energy_df['architecture'] =='Core i7-5650U' )].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu_mac =  list(energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu_mac = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')
y_etpu_mac = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')


x_cpu_ws = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
x_etpu_ws = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')

names_cpu_ws =  list(energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu_ws =  list(energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu_ws = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')
y_etpu_ws = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['accuracy_k1'].aggregate('mean')


fig = go.Figure()
for i in range(x_cpu_mac.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu_mac[i]], y=[y_cpu_mac[i]],
                        mode='markers',
                        name=str(names_cpu_mac[i]),
                        marker = dict(color = light_red_colour)))
for i in range(x_etpu_mac.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu_mac[i]], y=[y_etpu_mac[i]],
                        mode='markers',
                        name=str(names_etpu_mac[i]),
                        marker = dict(color = light_orange_colour)))
    
for i in range(x_cpu_ws.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu_ws[i]], y=[y_cpu_ws[i]],
                        mode='markers',
                        name=str(names_cpu_ws[i]),
                        marker = dict(color = red_colour)))
for i in range(x_etpu_ws.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu_ws[i]], y=[y_etpu_ws[i]],
                        mode='markers',
                        name=str(names_etpu_ws[i]),
                        marker = dict(color = orange_colour)))
fig.update_layout(
    xaxis_title="Energieverbrauch Ws",
    yaxis_title="Accuracy K=1",
)

fig.show()


## Running Time vs Power Draw

In [113]:

energy_df = data[data['approx_USB_power_draw'].notna() ]
x_cpu_mac = energy_df[(energy_df['backend'] == 'tf_cpu' )& (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu_mac = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['running_time'].aggregate('mean')

names_cpu_mac =  list(energy_df[(energy_df['backend'] == 'tf_cpu' )& (energy_df['architecture'] =='Core i7-5650U' )].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu_mac =  list(energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu_mac = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
y_etpu_mac = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-5650U')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')


x_cpu_ws = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['running_time'].aggregate('mean')
x_etpu_ws = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['running_time'].aggregate('mean')

names_cpu_ws =  list(energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'], group_keys=True).groups.keys())
names_etpu_ws =  list(energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'], group_keys=True).groups.keys())

#, c = createColorSequence(energy_df)
y_cpu_ws = energy_df[(energy_df['backend'] == 'tf_cpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')
y_etpu_ws = energy_df[(energy_df['backend'] == 'tflite_edgetpu') & (energy_df['architecture'] =='Core i7-6700')].groupby(['model','backend'])['approx_USB_power_draw'].aggregate('mean')


fig = go.Figure()
for i in range(x_cpu_mac.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu_mac[i]], y=[y_cpu_mac[i]],
                        mode='markers',
                        name=str(names_cpu_mac[i]),
                        marker = dict(color = light_red_colour)))
for i in range(x_etpu_mac.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu_mac[i]], y=[y_etpu_mac[i]],
                        mode='markers',
                        name=str(names_etpu_mac[i]),
                        marker = dict(color = light_orange_colour)))
    
for i in range(x_cpu_ws.shape[0]):
# Add traces
    fig.add_trace(go.Scatter(x=[x_cpu_ws[i]], y=[y_cpu_ws[i]],
                        mode='markers',
                        name=str(names_cpu_ws[i]),
                        marker = dict(color = red_colour)))
for i in range(x_etpu_ws.shape[0]):
    fig.add_trace(go.Scatter(x=[x_etpu_ws[i]], y=[y_etpu_ws[i]],
                        mode='markers',
                        name=str(names_etpu_ws[i]),
                        marker = dict(color = orange_colour)))
fig.update_layout(
    xaxis_title="Running Time",
    yaxis_title="Approx. Power Draw",
)

fig.show()
